## NDVI computation on Brown University cluster (Oscar).

Use the documentation to supplement this guide https://docs.ccv.brown.edu/oscar/getting-started.

### Objectives

1. Create a parallel processor 'job'and Python environment .
1. Modify landsatexplore.py from Week09 and implement a computation of NDVI using .
1. Use the Oscar cluster to split this calculation amongst the processors and peform the computation for all the satellite images in your items list.
1. Observe whether there is a seasonal difference in the NDVI for the RI/New England region.

**NOTE:** This exercise uses cumulative concepts from throughout the course, including use of list and numpy arrays, creation of modules, reading and writing files, using Pandas dataframes, and unix commands. Refer to your code from previous weeks and to the online library documentation as needed.

### Step 1:  Add NDVI computation to your previous `landsatexplore.py` code from Week09. 

The [Normalized Difference Vegetation Index](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) can be computed directly from two frequency bands in the Landsat satellite data - the Near Infrared band (NIR) and the visible red (Red) spectral reflectance:

$$ NDVI = \frac{NIR - Red}{NIR + Red} $$

Below is a list of modifications you should make to your `landsatexplore.py` script.  Give it a new name, like `lsexplore_NDVI.py` or similar.

**NOTE**: The landsatxplore API that we used in Week09 was very slow for downloading the Landsat files.  I have downloaded approximately 50 files and stored them in a location where you can load them.  The files are stored in `/users/urihpc25/LT05gz/`.  You can use this path and shell scripting wildcards to capture a list of e.g. all the files that end in `_B4.TIF` or `.tar.gz` and then feed that list to the dask client.map() function.

1. Delete, or comment out all usage of landsatexplore from Week09. This script will do the NDVI computation using the cluster job and save the results in a Pandas dataframe, nothing more.
1. Put all the code of the script inside this `if` statement. This ensures that the cluster resource request will be compiled at runtime before the computations begin:
~~~
if __name__ == '__main__':
~~~
Don't forget to indent
1. Add the codeblock from coreclock.py that requests and uses the requested Cluster resources.
1. Define a module to compute the NDVI with the following operations.  Your module should take a single file name from the landsat `scenes` list and it should return the average NDVI value from that item.

~~~
# Use skimage.imread() to load the NIR band (Band 4 on Landsat 5 images).  
# Do the same for the Red band (Band 3).
# Remember you can subindex the filename because it is a string and use it to load e.g. the 
# _B4.TIF and _B3.TIF files.
B4 = skimage.imread(<filepath>+...)
# Compute the NDVI as (nir-red)/(nir+red).  Note that in some cases, the denominator in this equation can be zero,
# leading to infinite or undefined values. Use e.g. np.where(), in order to identify the undefined values.
denom = np.add(B4,B3)
# Replace all undefined values with NaNs.
denom = np.where(...)

# Convert the .TIF arrays into dask arrays using dask.array.from_array().  Here you will specify a chunksize.

# Use dask.array functions to compute and return the average NDVI value from each satellite scene.

av_ndvi = ....

# Your function should return the average ndvi value for this Landsat image.
return av_ndvi

~~~

6. Add the code block from coreclock.py that uses f = client.map() to distribute the computation amongst the cores and displays the progress.  f = client.map() will take your module name and the satellite items list as inputs.

6. Write a `for` loop to extract the date stamp from the scenes list of images, and put it into a separate list array, called 'NDVI_time' or similar.  Refer to [this page on Landsat naming conventions](https://www.usgs.gov/faqs/what-naming-convention-landsat-collections-level-1-scenes?qt-news_science_products=0#qt-news_science_products) to determine which subset of the name contains the image date. 

6. Extract the list of Average NVDI computations from your module using client.gather().
~~~
AvNDVI_list = client.gather(f)
~~~
8. Make a Pandas dataframe with a 'time' column and an 'NDVI' column.  Put NDVI_time and AvNDVI_list into the Pandas dataframe (they should be the same length).

9. Save the Pandas dataframe for download. I recommend using df.to_csv('file.csv');  Depending on which you choose, you then you can read the file with df = pd.read_pickle() or df = read_csv();  



### Step 2:  Download and post-process your results.

1. Connect to Oscar with sftp as you have before.   Use sftp to transfer your Pandas dataframe

~~~
$ sftp guest###@ssh.ccv.brown.edu
$ lcd path/to/home/dir
$ cd path/to/results/file
$ get file.csv
~~~

2. Write a short script as .py or .ipynb to load and graph the NDVI mean value as a function of time.
3. Make a plot of the NDVI index with time, similar to the one below.
4. You can improve the rendering of dates on the y-axis using the mdates formatter:
~~~
import matplotlib.dates as mdates
# read in .csv...
# 
# Set index to datetime for rolling mean computation
ndv.set_index('time',inplace=True)
# Compute rolling mean. This gives a 60D running average.
ndv_rm = ndv.rolling('360D').mean();
# Plot....
# f, ax = plt.subplots(figsize=(10,5))
# 
#Display only year and month on x-axis.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m')) 
~~~

#### Summary of results:

<img src="https://bloose.github.io/data_prototyping_scientific_computing/images/NDVI.png" width="800"/>

**Results**
* I found 61 individual satellite items during a 10 year period from 2002 to 2012.
* The analysis took approximately 37 seconds to complete with 5 cores and 150 GB requested. This may vary, depending on how many users have jobs running at the same time and what priority those have.
* The NDVI index appears to vary a lot - possibly decreasing in winter months; lower NDVI suggests lower vegetation cover.
* Many factors can cause a change of NDVI including cloud cover or other details that we have not examined in the imagery.
* This exercise was mostly intended to lead us through the usage of computing on a remote cluster.

**Caveats**
1. Normal NDVI is supposed to scale between -1 and 1.  The raw values range from 0 to 255 digital numbers. We did not scale them, but I don't think this will skew the results as we are looking at relative changes.
1. Each image does not cover the same area. Before comparing regions, we should crop each image to the same area, but we leave this task for another iteration.
1. We did not consider how to account for cloud cover.
1. We did no benchmarking or cross-checking of these results, so they may be completely bogus.


### Step 3: What to turn in?

1. How long did your code take to process? 
1. Your modified version of landsatexplore.py from Week09, reflecting all the new code.
1. Modify this .ipynb to include your results from the NDVI calculation and summary figure.
1. Include the figure of NDVI over time, similar to the one above.